open-meteo.com provides hourly weather data, with high resolution. Hourly updates. predictions 16 days into to the future, as well as historical data.. Here the features are explored...

You can access past weather data dating back to 1940 with the historical weather API 
offered. *However, there is a 5-day delay in the data*. If you want information for the most recent days, you can use the forecast API and adjust the Past Days setting.

The strongest correlation to PV (photovoltaic) production is generally found with the following historical weather features:
	1.	Diffuse Horizontal Irradiance (DHI)
	2.	Direct Normal Irradiance (DNI) – This measures the amount of solar radiation received per unit area directly from the sun at a normal (perpendicular) angle. It is crucial for systems with solar tracking but less relevant for fixed panels.
	3.	Shortwave Solar Radiation (GHI) – Global Horizontal Irradiance (GHI) is the total solar radiation received on a horizontal surface and is commonly used in PV modeling.

    Additionally:
     Cloud cover Total, 
     Windspeed (10m) - due to cooling effect on PVs & wind energy production, 
     Temperature (2 m) - due to effect on PV efficiency and consumption level, 
     wind speed (100 m) for wind energy production

     are collected for De Bilt, Eelde (Groningen), Vlissingen, Sittard.
     for period 2022-2024.

In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [52.12949, 53.214469, 51.455677, 50.998299],
	"longitude": [5.20514, 6.566481, 3.576488, 5.86291],
	"start_date": "2022-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "cloud_cover", "wind_speed_10m", "shortwave_radiation", "diffuse_radiation", "direct_normal_irradiance"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(3).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(4).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.12653732299805°N 5.138121604919434°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  cloud_cover  wind_speed_10m  \
0     2022-01-01 00:00:00+00:00         10.4565        100.0       17.713316   
1     2022-01-01 01:00:00+00:00         10.8065        100.0       16.199999   
2     2022-01-01 02:00:00+00:00         10.5065         82.0       18.000000   
3     2022-01-01 03:00:00+00:00         10.2565         47.0       16.323528   
4     2022-01-01 04:00:00+00:00         10.1565        100.0       16.563911   
...                         ...             ...          ...             ...   
26299 2024-12-31 19:00:00+00:00          5.6565        100.0       24.541468   
26300 2024-12-31 20:00:00+00:00          6.4565        100.0       24.817816   
26301 2024-12-31 21:00:00+00:00          7.2065        100.0       25.750067   
26302 2024-12-31 22:00:00+00:00          7.7065        100.0       27.103003

In [4]:
df_size = hourly_dataframe.shape
print(f"The size of the dataframe is: {df_size}")

The size of the dataframe is: (26304, 7)


In [5]:
# Calculate the memory usage of the dataframe in bytes
memory_usage_bytes = hourly_dataframe.memory_usage(deep=True).sum()

# Convert the memory usage to kilobytes
memory_usage_kb = memory_usage_bytes / 1024

print(f"The size of the dataframe in kilobytes is: {memory_usage_kb:.2f} KB")

The size of the dataframe in kilobytes is: 822.13 KB


In [6]:
# Generate descriptive statistics for all columns in the dataframe
descriptive_stats = hourly_dataframe.describe()
print(descriptive_stats)

       temperature_2m   cloud_cover  wind_speed_10m  shortwave_radiation  \
count    26304.000000  26304.000000    26304.000000         26304.000000   
mean        11.558265     68.745171       14.702046           131.648270   
std          6.358396     38.889961        7.269707           204.015564   
min         -6.693500      0.000000        0.000000             0.000000   
25%          7.056500     31.000000        9.144637             0.000000   
50%         11.306500     97.000000       13.608762             5.000000   
75%         16.156500    100.000000       18.998316           201.250000   
max         34.256500    100.000000       57.785324           884.000000   

       diffuse_radiation  direct_normal_irradiance  
count       26304.000000              26304.000000  
mean           53.949970                152.752487  
std            76.681427                246.064987  
min             0.000000                  0.000000  
25%             0.000000                  0.000000